# Data_Preprocessing_Balanced

## Import Necessary Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # For optional plotting during EDA, can be commented out
import seaborn as sns         # For optional plotting during EDA, can be commented out
from sklearn.utils import resample # For undersampling

## Configuration

In [7]:
N_ROWS_TO_LOAD = 100000  # Load 100k rows from raw data
CSV_FILE_PATH = "brewery_data_complete_extended.csv"
OUTPUT_CSV_PATH = "preprocessed_brewery_data_balanced.csv"

## Define column names

In [10]:
FEATURE_COLUMNS = ['Gravity', 'Alcohol_Content', 'pH_Level', 'Moisture_Content', 'Temperature']
BRIX_COLUMN_NAME = 'Gravity'         # This is the original column name for Specific Gravity
ALCOHOL_COLUMN_NAME = 'Alcohol_Content'
PH_COLUMN_NAME = 'pH_Level'
MOISTURE_COLUMN_NAME = 'Moisture_Content'
TEMP_COLUMN_NAME = 'Temperature'
TARGET_COLUMN_NAME = 'Fermentation_Status'

## For reproducibility

In [13]:
np.random.seed(42)

## Load Raw Data

In [16]:
print(f"--- Loading Raw Data from: {CSV_FILE_PATH} ---")
try:
    df = pd.read_csv(CSV_FILE_PATH, nrows=N_ROWS_TO_LOAD)
    print(f"Successfully loaded {len(df)} rows.")
except FileNotFoundError:
    print(f"ERROR: File '{CSV_FILE_PATH}' not found. Creating a dummy DataFrame for demonstration.")
    data = {
        'Batch_ID': range(N_ROWS_TO_LOAD),
        BRIX_COLUMN_NAME: np.random.uniform(1.020, 1.090, N_ROWS_TO_LOAD), # SG
        ALCOHOL_COLUMN_NAME: np.random.uniform(3.0, 8.0, N_ROWS_TO_LOAD),
        PH_COLUMN_NAME: np.random.uniform(4.0, 6.0, N_ROWS_TO_LOAD),
        TEMP_COLUMN_NAME: np.random.uniform(10.0, 30.0, N_ROWS_TO_LOAD),
    }
    df = pd.DataFrame(data)
    print("Dummy DataFrame created.")

--- Loading Raw Data from: brewery_data_complete_extended.csv ---
Successfully loaded 100000 rows.


## Simulate Moisture Content (if not present)

In [19]:
if MOISTURE_COLUMN_NAME not in df.columns:
    print(f"\n--- Simulating '{MOISTURE_COLUMN_NAME}' ---")
    df[MOISTURE_COLUMN_NAME] = np.random.uniform(low=85.0, high=95.0, size=len(df)) - \
                               (df[ALCOHOL_COLUMN_NAME] * 0.1 if ALCOHOL_COLUMN_NAME in df.columns else 0)
    df[MOISTURE_COLUMN_NAME] = df[MOISTURE_COLUMN_NAME].round(2)
    # Add some NaNs for imputation demonstration later
    nan_indices = df.sample(frac=0.05, random_state=42).index
    df.loc[nan_indices, MOISTURE_COLUMN_NAME] = np.nan
else:
    print(f"\n--- Using existing '{MOISTURE_COLUMN_NAME}' column ---")



--- Simulating 'Moisture_Content' ---


## Convert Specific Gravity to Brix

In [22]:
if BRIX_COLUMN_NAME in df.columns:
    print(f"\n--- Converting '{BRIX_COLUMN_NAME}' (Specific Gravity) to Brix scale ---")
    df[BRIX_COLUMN_NAME] = pd.to_numeric(df[BRIX_COLUMN_NAME], errors='coerce')
    df = df.dropna(subset=[BRIX_COLUMN_NAME]) # Drop rows where Gravity couldn't be converted to numeric

    # Replace 0 with NaN before division to avoid errors and handle them
    df[BRIX_COLUMN_NAME] = df[BRIX_COLUMN_NAME].replace(0, np.nan)
    df.dropna(subset=[BRIX_COLUMN_NAME], inplace=True) # Remove rows where Gravity was 0

    if not df.empty: # Proceed only if DataFrame is not empty after NaN drops
        sg = df[BRIX_COLUMN_NAME]
        # Your provided formula for Brix from SG
        df[BRIX_COLUMN_NAME] = (28.14203 * sg) + 202.4427 - (230.5870 / sg)
        df[BRIX_COLUMN_NAME] = df[BRIX_COLUMN_NAME].round(2)
        print(f"Conversion complete. Column '{BRIX_COLUMN_NAME}' now contains Brix values.")
        # print(f"First 5 converted Brix values:\n{df[BRIX_COLUMN_NAME].head()}")
    else:
        print(f"DataFrame became empty after trying to clean '{BRIX_COLUMN_NAME}' for Brix conversion. Halting.")
        # raise SystemExit("Halting due to data issue in Brix conversion.") # Optional: halt execution
else:
    print(f"Warning: Column '{BRIX_COLUMN_NAME}' (expected for Gravity) not found for Brix conversion.")



--- Converting 'Gravity' (Specific Gravity) to Brix scale ---
Conversion complete. Column 'Gravity' now contains Brix values.


## Ensure all feature columns exist, fill with NaN if created new

In [25]:
for col in FEATURE_COLUMNS:
    if col not in df.columns:
        print(f"Warning: Feature column '{col}' was not in the original CSV or created. Adding as NaN.")
        df[col] = np.nan

## Select only the feature columns we intend to process further + any ID columns if needed later
## For simplicity, we'll just work with the defined FEATURE_COLUMNS from now on

In [28]:
df_features = df[FEATURE_COLUMNS].copy()

## Handle Data Types and Coercion for all feature columns

In [31]:
print(f"\n--- Handling Data Types for Feature Columns ---")
def clean_feature_value(value):
    if isinstance(value, str):
        for delim in ['-', '/', ' ', '_']:
            if delim in value: value = value.split(delim)[0]
        value = ''.join(c for c in str(value) if c.isdigit() or c == '.')
        parts = value.split('.')
        if len(parts) > 2: value = parts[0] + '.' + parts[1]
    try: return float(value)
    except (ValueError, TypeError): return np.nan

for col in FEATURE_COLUMNS:
    if col in df_features.columns:
        if df_features[col].dtype == 'object': # If it's still an object type
            print(f"Applying string cleaning to object column {col}...")
            df_features[col] = df_features[col].apply(clean_feature_value)
        # Final explicit conversion to numeric, coercing errors
        df_features[col] = pd.to_numeric(df_features[col], errors='coerce')


--- Handling Data Types for Feature Columns ---


## Handle Missing Values (Median Imputation)

In [34]:
print(f"\n--- Handling Missing Values (Median Imputation) ---")
for col in FEATURE_COLUMNS:
    if col in df_features.columns and df_features[col].isnull().any():
        median_val = df_features[col].median()
        df_features[col] = df_features[col].fillna(median_val)
        print(f"Imputed NaNs in '{col}' with median {median_val:.2f}.")
# Drop any rows that are still all NaN across features (should be rare after imputation)
df_features.dropna(how='all', inplace=True)
df = df.loc[df_features.index].copy() # Align original df if rows were dropped (e.g. if a whole feature column was NaN)
df[FEATURE_COLUMNS] = df_features[FEATURE_COLUMNS] # Update df with cleaned features


--- Handling Missing Values (Median Imputation) ---
Imputed NaNs in 'Moisture_Content' with median 89.47.


## IQR Outlier Capping

In [37]:
print(f"\n--- Applying IQR Outlier Capping ---")
for col in FEATURE_COLUMNS:
    if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
        df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
print("IQR outlier capping complete.")
print("\nDescriptive statistics after outlier capping:")
print(df[FEATURE_COLUMNS].describe())


--- Applying IQR Outlier Capping ---
IQR outlier capping complete.

Descriptive statistics after outlier capping:
             Gravity  Alcohol_Content       pH_Level  Moisture_Content  \
count  100000.000000    100000.000000  100000.000000     100000.000000   
mean       13.521659         5.251379       5.000560         89.466125   
std         3.398580         0.433797       0.288597          2.812281   
min         7.560000         4.500014       4.500001         84.410000   
25%        10.590000         4.873972       4.751442         87.090000   
50%        13.560000         5.252499       5.001072         89.470000   
75%        16.480000         5.627443       5.251072         91.830000   
max        19.330000         5.999956       5.499992         94.550000   

         Temperature  
count  100000.000000  
mean       20.019249  
std         2.886794  
min        15.000089  
25%        17.518969  
50%        20.030475  
75%        22.505320  
max        24.999868  


## Simulate Target Variable

In [40]:
print(f"\n--- Simulating Target Variable '{TARGET_COLUMN_NAME}' ---")


--- Simulating Target Variable 'Fermentation_Status' ---


## Ensure key columns for simulation are numeric and handle any NaNs that might stop the rule

In [43]:
df[ALCOHOL_COLUMN_NAME] = pd.to_numeric(df[ALCOHOL_COLUMN_NAME], errors='coerce')
df[BRIX_COLUMN_NAME] = pd.to_numeric(df[BRIX_COLUMN_NAME], errors='coerce') # Already Brix values
df.dropna(subset=[ALCOHOL_COLUMN_NAME, BRIX_COLUMN_NAME], inplace=True) # Critical for reliable simulation

if df.empty:
    print("ERROR: DataFrame is empty after ensuring simulation columns are clean. Cannot create target.")
else:
    # **ADJUST THIS RULE BASED ON YOUR BRIX SCALE AND DESIRED BALANCE BEFORE UNDERSAMPLING**
    # Example: Target '1' (Ready) if Alcohol > 5.2 AND Brix (in 'Gravity' col) < 8.0
    # This rule needs to generate a reasonable number of '1's to make balancing meaningful.
    df[TARGET_COLUMN_NAME] = 0
    ready_condition = (df[ALCOHOL_COLUMN_NAME] > 5.2) & (df[BRIX_COLUMN_NAME] < 8.0)
    df.loc[ready_condition, TARGET_COLUMN_NAME] = 1
    
    print("\nTarget variable distribution BEFORE balancing:")
    print(df[TARGET_COLUMN_NAME].value_counts(normalize=True))
    print(df[TARGET_COLUMN_NAME].value_counts())


Target variable distribution BEFORE balancing:
Fermentation_Status
0    0.98124
1    0.01876
Name: proportion, dtype: float64
Fermentation_Status
0    98124
1     1876
Name: count, dtype: int64


## Balance Dataset (Undersampling Majority)

In [46]:
print(f"\n--- Balancing Dataset for '{TARGET_COLUMN_NAME}' ---")
if TARGET_COLUMN_NAME not in df.columns or df[TARGET_COLUMN_NAME].nunique() < 2 :
     print(f"Target column '{TARGET_COLUMN_NAME}' not found or has only one class. Skipping balancing.")
     df_balanced = df.copy() # Use the df as is if target is problematic
elif df.empty:
    print("DataFrame is empty before balancing. Skipping.")
    df_balanced = df.copy()
else:
    df_majority = df[df[TARGET_COLUMN_NAME] == 0]
    df_minority = df[df[TARGET_COLUMN_NAME] == 1]

    if len(df_minority) == 0:
        print("ERROR: Minority class (1) has 0 samples. Cannot balance by undersampling. Saving as is.")
        df_balanced = df.copy()
    elif len(df_majority) == 0:
        print("ERROR: Majority class (0) has 0 samples. Data is already 100% minority. Saving as is.")
        df_balanced = df.copy()
    elif len(df_majority) == len(df_minority):
        print("Dataset is already balanced.")
        df_balanced = df.copy()
    else:
        print(f"Undersampling majority class (0) from {len(df_majority)} to match minority class (1) of {len(df_minority)} samples.")
        df_majority_downsampled = resample(df_majority,
                                         replace=False,
                                         n_samples=len(df_minority),
                                         random_state=42)
        df_balanced = pd.concat([df_majority_downsampled, df_minority])
        
        print("\nTarget variable distribution AFTER balancing:")
        print(df_balanced[TARGET_COLUMN_NAME].value_counts(normalize=True))
        print(df_balanced[TARGET_COLUMN_NAME].value_counts())
        print(f"Total samples in balanced dataset: {len(df_balanced)}")


--- Balancing Dataset for 'Fermentation_Status' ---
Undersampling majority class (0) from 98124 to match minority class (1) of 1876 samples.

Target variable distribution AFTER balancing:
Fermentation_Status
0    0.5
1    0.5
Name: proportion, dtype: float64
Fermentation_Status
0    1876
1    1876
Name: count, dtype: int64
Total samples in balanced dataset: 3752


## Shuffle the balanced (or original if balancing failed/skipped) dataset

In [49]:
df_to_save = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"\nDataFrame to be saved has {len(df_to_save)} rows.")


DataFrame to be saved has 3752 rows.


## Save Preprocessed and Balanced Data

In [52]:
print(f"\n--- Saving Preprocessed and Balanced Data to: {OUTPUT_CSV_PATH} ---")
columns_to_save = FEATURE_COLUMNS + [TARGET_COLUMN_NAME]
# Ensure all columns exist in df_to_save before trying to select them
final_columns_present = [col for col in columns_to_save if col in df_to_save.columns]

if not df_to_save.empty and set(columns_to_save) <= set(df_to_save.columns):
    df_final_output = df_to_save[final_columns_present]
    try:
        df_final_output.to_csv(OUTPUT_CSV_PATH, index=False)
        print(f"\nSuccessfully saved data with {len(df_final_output)} rows and columns {df_final_output.columns.tolist()} to '{OUTPUT_CSV_PATH}'.")
        print("\nFirst 5 rows of the saved data:")
        print(df_final_output.head())
        print("\nFinal distribution in saved file:")
        print(df_final_output[TARGET_COLUMN_NAME].value_counts(normalize=True))
        if df_final_output[TARGET_COLUMN_NAME].nunique() < 2:
            print("WARNING: The saved file still contains only one class for the target variable!")
    except Exception as e:
        print(f"Error saving preprocessed data: {e}")
else:
    if df_to_save.empty:
        print("\nCould not save: DataFrame is empty.")
    else:
        print(f"\nCould not save: Not all required columns ({columns_to_save}) are present in the DataFrame to be saved.")
        print(f"Columns available: {df_to_save.columns.tolist()}")


--- Saving Preprocessed and Balanced Data to: preprocessed_brewery_data_balanced.csv ---

Successfully saved data with 3752 rows and columns ['Gravity', 'Alcohol_Content', 'pH_Level', 'Moisture_Content', 'Temperature', 'Fermentation_Status'] to 'preprocessed_brewery_data_balanced.csv'.

First 5 rows of the saved data:
   Gravity  Alcohol_Content  pH_Level  Moisture_Content  Temperature  \
0     7.60         5.278893  5.365859             88.59    17.724930   
1    16.33         5.758956  5.125840             84.89    15.695592   
2     7.69         5.563351  4.732960             93.72    23.258060   
3    11.56         4.658732  5.013984             86.96    16.272746   
4     7.98         5.411103  4.784614             90.23    23.936993   

   Fermentation_Status  
0                    1  
1                    0  
2                    1  
3                    0  
4                    1  

Final distribution in saved file:
Fermentation_Status
1    0.5
0    0.5
Name: proportion, dtype